In [1]:
import glob
from PIL import Image
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from datetime import date
from datetime import datetime

D:\programming\Anaconda1\envs\NN\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
import tensorflow as tf
import numpy as np
import argparse
import math
import os
from matplotlib.image import imread
from scipy.misc.pilutil import imresize, imsave
import matplotlib.pyplot as plt
import cv2

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Flatten, Dropout, Input
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.datasets import mnist
from keras import initializers

import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def build_generator():
    
        model = Sequential()
        model.add(Dense(128 * 16 * 8, input_dim = latent_dim))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Reshape((8, 8, 256)))
        
        model.add(Conv2DTranspose(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(256, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(128, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(64, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(32, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(16, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(8, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(img_channels, filter_size_g, strides=(1,1), padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        return model

In [ ]:
def build_discriminator(num_classes):

        model = Sequential()
        
        model.add(Conv2D(64, kernel_size=filter_size_d, strides = (2,2), input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256,  kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        
        model.summary()
        
        img = Input(shape=img_shape)

        features = model(img)

        validity = Dense(1)(features)
        valid = Activation('sigmoid')(validity)
        
        label1 = Dense(512)(features)
        lrelu1 = LeakyReLU(alpha=0.2)(label1)
        label2 = Dense(256)(label1)
        lrelu2 = LeakyReLU(alpha=0.2)(label2)
        label3 = Dense(num_classes)(label2)
        label = Activation('softmax')(label3)
        
        return Model(img, valid), Model(img, label)

In [ ]:
def generator_containing_discriminator(g, d, d_label):
    noise = Input(shape=(latent_dim,))
    img = g(noise)
    d.trainable = False
    d_label.trainable = False
    valid, target_label = d(img), d_label(img)
    
    
    return Model(noise, [valid, target_label])

In [ ]:
def save_imgs(epoch, gen):
        count = 5
        noise = np.random.uniform(-1, 1, size=(count, latent_dim))
        gen_imgs = gen.predict(noise)

        gen_imgs = 127.5 * gen_imgs + 127.5
        
        if gen_imgs.shape[3] == 1:
            gen_imgs = gen_imgs[:,:,:,0]
        
        for i in range(count):
            cv2.imwrite(os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, i), gen_imgs[i])

In [ ]:
def get_images(indices, data, batch_size):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    for i in range(batch_size):
        if color_mode == 'grayscale':
            temp_img = cv2.imread(data[indices[i]], 0)
            X_train[i,:,:,0] = temp_img
        else:
            temp_img = cv2.imread(data[indices[i]])
            X_train[i] = temp_img
    X_train = (X_train - 127.5)/127.5
    return X_train

In [ ]:
def get_images_classes(batch_size, data):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size)
    choice_arr = np.random.randint(0, len(data), batch_size)
    for i in range(batch_size):
        rand_number = np.random.randint(0, len(data[choice_arr[i]]))
        temp_img = cv2.imread(data[choice_arr[i]][rand_number])
        
        
        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]
    

    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def get_images_one_class(batch_size, data, class_target):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_label = np.zeros(batch_size)
    '''choice_arr = np.random.randint(0, len(data[class_target]), batch_size)
    for i in range(batch_size):
        temp_img = cv2.imread(data[class_target][choice_arr[i]])

        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]'''
    
    for i in range(4):
        for j in range(4):
            temp_img = cv2.imread(data[i][j])
            
            X_train[4*i + j] = temp_img
            y_label[4*i + j] = i
            
    X_train = (X_train - 127.5)/127.5
    return X_train, y_label

In [ ]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1], img_channels),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, :,]
    return image

In [ ]:
def get_data():
    styles_folder = os.listdir(path=os.getcwd() + "\\new256_images\\")
    num_styles = len(styles_folder)
    data = []
    for i in range(num_styles):
        data.append(glob.glob(os.getcwd() + '\\new256_images\\' + styles_folder[i] + '\\*'))
    return data, num_styles

In [ ]:
def train_another(epochs = 100, BATCH_SIZE = 4, weights = False, month_day = ''):

    data, num_styles = get_data()
    
    
    generator = build_generator()
    discriminator, d_label = build_discriminator(num_styles)
    
    discriminator.compile(loss=losses[0], optimizer=d_optim)
    d_label.compile(loss=losses[1], optimizer=d_optim)
    generator.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    
    if weights:
        generator.load_weights(os.getcwd() + '/' + month_day + ' gen_weights.h5')
        discriminator.load_weights(os.getcwd() + '/' + month_day + ' dis_weights.h5')
        d_label.load_weights(os.getcwd() + '/' + month_day + ' dis_label_weights.h5')
        
    
    dcgan = generator_containing_discriminator(generator, discriminator, d_label)
    
    
    dcgan.compile(loss=losses[0], optimizer=g_optim)
    
    discriminator.trainable = True
    d_label.trainable = True
    
    
    '''(X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))'''
    
    
    for epoch in range(epochs):
        for index in range(int(15000/BATCH_SIZE)):
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
                
            #real_images, real_labels = get_images_classes(batch_size=BATCH_SIZE, data = data)
            
            label = index % num_styles
            
            real_images, real_labels = get_images_classes(BATCH_SIZE, data)
            
            #real_images += np.random.normal(size = img_shape, scale= 0.1)
            
            generated_images = generator.predict(noise)
            
            if index % 2 == 0:
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + real_labels, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                discriminator.trainable = False
                d_loss.append(d_label.train_on_batch(X, y_classif))
                print("epoch %d batch %d d_loss : %f, label_loss: %f" % (epoch, index, d_loss[0], d_loss[1]))
                
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss))
            else:
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss))
                
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + real_labels, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                discriminator.trainable = False
                d_loss.append(d_label.train_on_batch(X, y_classif))
                
                print("epoch %d batch %d d_loss : %f, label_loss: %f" % (epoch, index, d_loss[0], d_loss[1]))
            
            
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
            
            discriminator.trainable = False
            d_label.trainable = False
            
            y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + 1/num_styles, num_styles)
            y = np.random.rand(BATCH_SIZE) * 0.3
            
            g_loss = dcgan.train_on_batch(noise, [y, y_classif])
            
            d_label.trainable = True
            discriminator.trainable = True
            
            print("epoch %d batch %d g_loss : %f, label_loss: %f" % (epoch, index, g_loss[0], g_loss[1]))
            
            if index % 50 == 0:
                        image = combine_images(generated_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, index), image)
                        image = combine_images(real_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d_data.png' % (epoch, index), image)
                        
        if epoch % 5 == 0:
            
            
            date_today = date.today()
            
            
            
            month, day = date_today.month, date_today.day
            
            # Генерируем описание модели в формате json
            d_json = discriminator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "/%d.%d dis_model.json" % (day, month), "w")
            json_file.write(d_json)
            json_file.close()
            
            # Генерируем описание модели в формате json
            d_l_json = d_label.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "/%d.%d dis_label_model.json" % (day, month), "w")
            json_file.write(d_l_json)
            json_file.close()
            
            # Генерируем описание модели в формате json
            gen_json = generator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "/%d.%d gen_model.json" % (day, month), "w")
            json_file.write(gen_json)
            json_file.close()
            
            discriminator.save_weights(os.getcwd() + '/%d.%d ' + epoch + '_epoch dis_weights.h5' % (day, month))
            d_label.save_weights(os.getcwd() + '/%d.%d ' + epoch + '_epoch dis_label_weights.h5' % (day, month))
            generator.save_weights(os.getcwd() + '/%d.%d ' + epoch + '_epoch gen_weights.h5' % (day, month))

In [ ]:
img_rows = 256
img_cols = 256
img_channels = 3
img_shape = (img_rows, img_cols, img_channels)
latent_dim = 100
filter_size_g = (5,5)
filter_size_d = (5,5)
d_strides = (2,2)

color_mode = 'rgb'

losses = ['binary_crossentropy', 'categorical_crossentropy']


#g_optim = SGD(lr = 0.001, momentum=0.9, nesterov=True)
#d_optim = SGD(lr = 0.00025, momentum=0.9, nesterov=True)
g_optim = Adam(0.0002, beta_2 = 0.5)
d_optim = Adam(0.0002, beta_2 = 0.5)

In [ ]:
train_another(1000, 16, weights = True, month_day = '27.11')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16384)             1654784   
_________________________________________________________________
batch_normalization_1 (Batch (None, 16384)             65536     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16384)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 512)         3277312   
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 512)         2048      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 512)         0         
__________

D:\programming\Anaconda1\envs\NN\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch 0 batch 0 d_loss : 0.388204
epoch 0 batch 0 g_loss : 0.534445, label_loss: 0.461655
epoch 0 batch 1 d_loss : 0.351392
epoch 0 batch 1 d_loss : 0.309485, label_loss: 0.915225
epoch 0 batch 1 g_loss : 0.500522, label_loss: 0.420786
epoch 0 batch 2 d_loss : 0.420880, label_loss: 0.961532
epoch 0 batch 2 d_loss : 0.333750
epoch 0 batch 2 g_loss : 0.486878, label_loss: 0.398541
epoch 0 batch 3 d_loss : 0.300823
epoch 0 batch 3 d_loss : 0.368363, label_loss: 1.117343
epoch 0 batch 3 g_loss : 0.509614, label_loss: 0.431173
epoch 0 batch 4 d_loss : 0.337859, label_loss: 1.040391
epoch 0 batch 4 d_loss : 0.315413
epoch 0 batch 4 g_loss : 0.418230, label_loss: 0.350618
epoch 0 batch 5 d_loss : 0.306099
epoch 0 batch 5 d_loss : 0.353862, label_loss: 1.103593
epoch 0 batch 5 g_loss : 0.579888, label_loss: 0.507933
epoch 0 batch 6 d_loss : 0.351724, label_loss: 0.943890
epoch 0 batch 6 d_loss : 0.312846
epoch 0 batch 6 g_loss : 0.536273, label_loss: 0.468093
epoch 0 batch 7 d_loss : 0.303006


epoch 0 batch 55 g_loss : 0.590833, label_loss: 0.506740
epoch 0 batch 56 d_loss : 0.275505, label_loss: 0.886352
epoch 0 batch 56 d_loss : 0.323155
epoch 0 batch 56 g_loss : 0.521823, label_loss: 0.434266
epoch 0 batch 57 d_loss : 0.393435
epoch 0 batch 57 d_loss : 0.349431, label_loss: 0.900120
epoch 0 batch 57 g_loss : 0.522155, label_loss: 0.436000
epoch 0 batch 58 d_loss : 0.325302, label_loss: 0.915920
epoch 0 batch 58 d_loss : 0.354882
epoch 0 batch 58 g_loss : 0.561520, label_loss: 0.469633
epoch 0 batch 59 d_loss : 0.317029
epoch 0 batch 59 d_loss : 0.364636, label_loss: 0.928969
epoch 0 batch 59 g_loss : 0.499295, label_loss: 0.416476
epoch 0 batch 60 d_loss : 0.290393, label_loss: 1.176256
epoch 0 batch 60 d_loss : 0.370805
epoch 0 batch 60 g_loss : 0.442930, label_loss: 0.357562
epoch 0 batch 61 d_loss : 0.337199
epoch 0 batch 61 d_loss : 0.323803, label_loss: 0.922219
epoch 0 batch 61 g_loss : 0.480652, label_loss: 0.404815
epoch 0 batch 62 d_loss : 0.326196, label_loss: 0

epoch 0 batch 110 g_loss : 0.654752, label_loss: 0.561800
epoch 0 batch 111 d_loss : 0.336666
epoch 0 batch 111 d_loss : 0.303994, label_loss: 1.096661
epoch 0 batch 111 g_loss : 0.610139, label_loss: 0.518620
epoch 0 batch 112 d_loss : 0.345600, label_loss: 1.380768
epoch 0 batch 112 d_loss : 0.322857
epoch 0 batch 112 g_loss : 0.499169, label_loss: 0.409501
epoch 0 batch 113 d_loss : 0.312149
epoch 0 batch 113 d_loss : 0.341991, label_loss: 0.920702
epoch 0 batch 113 g_loss : 0.534805, label_loss: 0.454758
epoch 0 batch 114 d_loss : 0.322334, label_loss: 1.263766
epoch 0 batch 114 d_loss : 0.310908
epoch 0 batch 114 g_loss : 0.501697, label_loss: 0.414761
epoch 0 batch 115 d_loss : 0.347535
epoch 0 batch 115 d_loss : 0.337463, label_loss: 0.852616
epoch 0 batch 115 g_loss : 0.477127, label_loss: 0.398018
epoch 0 batch 116 d_loss : 0.381424, label_loss: 1.387515
epoch 0 batch 116 d_loss : 0.347919
epoch 0 batch 116 g_loss : 0.562515, label_loss: 0.482602
epoch 0 batch 117 d_loss : 0.3

...

epoch 4 batch 421 g_loss : 0.486973, label_loss: 0.413306
epoch 4 batch 422 d_loss : 0.295859, label_loss: 0.860348
epoch 4 batch 422 d_loss : 0.254342
epoch 4 batch 422 g_loss : 0.465681, label_loss: 0.392715
epoch 4 batch 423 d_loss : 0.378942
epoch 4 batch 423 d_loss : 0.339038, label_loss: 0.912966
epoch 4 batch 423 g_loss : 0.470645, label_loss: 0.403111
epoch 4 batch 424 d_loss : 0.345945, label_loss: 0.974059
epoch 4 batch 424 d_loss : 0.300371
epoch 4 batch 424 g_loss : 0.512947, label_loss: 0.435080
epoch 4 batch 425 d_loss : 0.310668
epoch 4 batch 425 d_loss : 0.306613, label_loss: 0.906337
epoch 4 batch 425 g_loss : 0.540876, label_loss: 0.466434
epoch 4 batch 426 d_loss : 0.301953, label_loss: 1.322719
epoch 4 batch 426 d_loss : 0.392612
epoch 4 batch 426 g_loss : 0.540057, label_loss: 0.463731
epoch 4 batch 427 d_loss : 0.363777
epoch 4 batch 427 d_loss : 0.323885, label_loss: 1.000153
epoch 4 batch 427 g_loss : 0.520603, label_loss: 0.446301
epoch 4 batch 428 d_loss : 0.3

epoch 4 batch 475 g_loss : 0.446393, label_loss: 0.389661
epoch 4 batch 476 d_loss : 0.298360, label_loss: 1.060501
epoch 4 batch 476 d_loss : 0.295389
epoch 4 batch 476 g_loss : 0.519896, label_loss: 0.463470
epoch 4 batch 477 d_loss : 0.315738
epoch 4 batch 477 d_loss : 0.374855, label_loss: 0.764740
epoch 4 batch 477 g_loss : 0.554809, label_loss: 0.501143
epoch 4 batch 478 d_loss : 0.353517, label_loss: 0.934015
epoch 4 batch 478 d_loss : 0.329650
epoch 4 batch 478 g_loss : 0.519538, label_loss: 0.467926
epoch 4 batch 479 d_loss : 0.313956
epoch 4 batch 479 d_loss : 0.349712, label_loss: 1.126176
epoch 4 batch 479 g_loss : 0.436304, label_loss: 0.389316
epoch 4 batch 480 d_loss : 0.316939, label_loss: 1.155713
epoch 4 batch 480 d_loss : 0.316090
epoch 4 batch 480 g_loss : 0.502794, label_loss: 0.453535
epoch 4 batch 481 d_loss : 0.336984
epoch 4 batch 481 d_loss : 0.362402, label_loss: 1.034607
epoch 4 batch 481 g_loss : 0.510491, label_loss: 0.461335
epoch 4 batch 482 d_loss : 0.3

epoch 4 batch 529 g_loss : 0.483281, label_loss: 0.433059
epoch 4 batch 530 d_loss : 0.381054, label_loss: 0.939686
epoch 4 batch 530 d_loss : 0.362562
epoch 4 batch 530 g_loss : 0.538673, label_loss: 0.485592
epoch 4 batch 531 d_loss : 0.296007
epoch 4 batch 531 d_loss : 0.348994, label_loss: 1.223770
epoch 4 batch 531 g_loss : 0.539621, label_loss: 0.487225
epoch 4 batch 532 d_loss : 0.320543, label_loss: 1.059759
epoch 4 batch 532 d_loss : 0.372980
epoch 4 batch 532 g_loss : 0.520674, label_loss: 0.464278
epoch 4 batch 533 d_loss : 0.359547
epoch 4 batch 533 d_loss : 0.337358, label_loss: 0.958186
epoch 4 batch 533 g_loss : 0.502930, label_loss: 0.446625
epoch 4 batch 534 d_loss : 0.296165, label_loss: 1.186019
epoch 4 batch 534 d_loss : 0.376693
epoch 4 batch 534 g_loss : 0.511149, label_loss: 0.451705
epoch 4 batch 535 d_loss : 0.317028
epoch 4 batch 535 d_loss : 0.369320, label_loss: 0.939935
epoch 4 batch 535 g_loss : 0.472649, label_loss: 0.426591
epoch 4 batch 536 d_loss : 0.3

epoch 4 batch 583 g_loss : 0.553369, label_loss: 0.485909
epoch 4 batch 584 d_loss : 0.408106, label_loss: 0.931616
epoch 4 batch 584 d_loss : 0.309716
epoch 4 batch 584 g_loss : 0.599504, label_loss: 0.529895
epoch 4 batch 585 d_loss : 0.304049
epoch 4 batch 585 d_loss : 0.356073, label_loss: 0.968497
epoch 4 batch 585 g_loss : 0.510222, label_loss: 0.434971
epoch 4 batch 586 d_loss : 0.349442, label_loss: 0.871001
epoch 4 batch 586 d_loss : 0.334328
epoch 4 batch 586 g_loss : 0.523293, label_loss: 0.453490
epoch 4 batch 587 d_loss : 0.374119
epoch 4 batch 587 d_loss : 0.287101, label_loss: 1.171951
epoch 4 batch 587 g_loss : 0.484164, label_loss: 0.414994
epoch 4 batch 588 d_loss : 0.369983, label_loss: 1.239577
epoch 4 batch 588 d_loss : 0.300506
epoch 4 batch 588 g_loss : 0.500107, label_loss: 0.430395
epoch 4 batch 589 d_loss : 0.369663
epoch 4 batch 589 d_loss : 0.321918, label_loss: 1.160479
epoch 4 batch 589 g_loss : 0.464491, label_loss: 0.393955
epoch 4 batch 590 d_loss : 0.3

epoch 4 batch 637 g_loss : 0.546313, label_loss: 0.492621
epoch 4 batch 638 d_loss : 0.322070, label_loss: 0.968027
epoch 4 batch 638 d_loss : 0.294421
epoch 4 batch 638 g_loss : 0.479874, label_loss: 0.422539
epoch 4 batch 639 d_loss : 0.382694
epoch 4 batch 639 d_loss : 0.353063, label_loss: 0.772718
epoch 4 batch 639 g_loss : 0.518482, label_loss: 0.460244
epoch 4 batch 640 d_loss : 0.310419, label_loss: 0.887485
epoch 4 batch 640 d_loss : 0.316890
epoch 4 batch 640 g_loss : 0.521602, label_loss: 0.468571
epoch 4 batch 641 d_loss : 0.333929
epoch 4 batch 641 d_loss : 0.355323, label_loss: 1.080978
epoch 4 batch 641 g_loss : 0.478013, label_loss: 0.417742
epoch 4 batch 642 d_loss : 0.318849, label_loss: 0.944033
epoch 4 batch 642 d_loss : 0.360698
epoch 4 batch 642 g_loss : 0.514134, label_loss: 0.462158
epoch 4 batch 643 d_loss : 0.352729
epoch 4 batch 643 d_loss : 0.390726, label_loss: 0.958921
epoch 4 batch 643 g_loss : 0.471817, label_loss: 0.414083
epoch 4 batch 644 d_loss : 0.3

In [ ]:
current_moment = datetime.now()

day, month = str(current_moment.day), str(current_moment.month)
hour, minute = str(current_moment.hour), str(current_moment.minute)

with open(os.getcwd() + '/Changes.txt', 'a') as f:
    clock =  (day + '.' + month + '    ' + hour + ':' + minute)
    f.write(clock + '...........g_optim -> 0.0003, d_optim->0.0004' + '\n')